In [1]:
import os
import pandas as pd
from pymongo import MongoClient
from typing import List, Dict, Set
from datetime import datetime
from dataclasses import dataclass, asdict
from pymongo.cursor import Cursor
from tqdm import tqdm

import dotenv
dotenv.load_dotenv(".env", override=True) # Defines MONGO_URI

True

In [21]:
def cursor_to_pandas(cursor):
    dfs = []
    for item in cursor:
        df = pd.DataFrame({"cve": item["cves"]})
        df["registry"] = item["registry"]
        df["repository"] = item["repository"]
        df["tag"] = item["tag"]
        df["scan_start"] = item["scan_start"]
        cve_df = pd.json_normalize(df["cve"])
        df = pd.concat([df.drop(columns=["cve"]), cve_df], axis=1)
        dfs.append(df)
    return pd.concat(dfs, axis=0, ignore_index=True)

In [25]:
client = MongoClient(os.environ["MONGO_URI"])
collection = client["gallery"]["cves"]

cursor = collection.find({"registry": "cgr.dev"})

df = cursor_to_pandas(cursor)

In [29]:
df.groupby(["repository"])["cves"].count()

KeyError: 'Column not found: cves'